In [ ]:
import xarray as xr
import numpy as np
from math import pi
import datetime as dt
import os
import sys
from pathlib import Path
from os.path import exists
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
#from pathlib import Path
from datetime import *; from dateutil.relativedelta import *
#from scipy.interpolate import make_interp_spline, BSpline
#from scipy.signal import savgol_filter
%matplotlib inline
sys.path.append('./subroutines/')
from read_gridded_data import get_monthly_data

boxes_names = ['NPC','NPC_biforcation','CalCur','Alaska1','Alaska2','CalCur1','CalCur3']

def get_center_kathleen(rlon):
    if rlon==-165:
        ds_lon = xr.open_dataset('F:/data/NASA_biophysical/timeseries_data/MonthlyBifurcationLat165.nc')
    if rlon==-135:
        ds_lon = xr.open_dataset('F:/data/NASA_biophysical/timeseries_data/MonthlyBifurcationLat135.nc')
    ds_lon['month']=np.ceil((ds_lon.year-ds_lon.year.astype('int'))*12).astype('int')
    ds_lon['iyear']=ds_lon.year.astype('int')
    ds_lon = ds_lon.assign_coords(yr=ds_lon.year)
    ds_lon = ds_lon.assign_coords(iyear=ds_lon.iyear)
    ds_lon = ds_lon.assign_coords(month=ds_lon.month)
    ds_out = ds_lon.copy(deep=True)    
    tt=[]
    for i in range(276):
        dtem=np.datetime64(str(ds_out.iyear[i].data)+'-'+str(ds_out.month[i].data).zfill(2))
        tt.append(dtem)
    ds_lon.close()
    ds_out['yr']=tt
    return ds_out

def get_center_chelle(rlon):
    adir_data='F:/data/NASA_biophysical/biforcation_figures_chelle/'
    if rlon==-165:
        ds=xr.open_dataset(adir_data+'biforcation_data165_monthly_aviso.nc')
    if rlon==-135:
        ds=xr.open_dataset(adir_data+'biforcation_data135_monthly_aviso.nc')
    ds.close()
    return ds


#trying to get into xarray properly
location_values = ['NPC','NPC_biforcation','CalCur','Alaska1','Alaska2','NPC-var','NPC-bi-var','CalCur1','CalCur3']
data = [-160,-130,-120,-130,-148,-160,-130,-122,-116]
lon1 = xr.DataArray(data, coords=[location_values], dims=['location'])
data = [-170,-140,-130,-140,-158,-170,-140,-132,-126]
lon2 = xr.DataArray(data, coords=[location_values], dims=['location'])
data = [43.5,48.5,37.5,54.5,54.5,43.5,48.5,42.5,32.5]
lat1 = xr.DataArray(data, coords=[location_values], dims=['location'])
data = [46,51,40,57,57,46,51,45,35]
lat2 = xr.DataArray(data, coords=[location_values], dims=['location'])
boxes = xr.Dataset({'lon_min':lon1,'lon_max':lon2,'lat_cen':lat1})

def get_mask():
    filelist=[]
    dir_data = 'F:/data/sst/cmc/CMC0.2deg/v2/data/'
    for filename in Path(dir_data).rglob('*CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'):
        filelist.append(filename)
    ds=xr.open_dataset(filelist[0])
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon').sortby('lat')
    ds = ds.resample(time='M',keep_attrs=True).mean(skipna=False,keep_attrs=True)
    mask=ds.mask.where(ds.mask[0,:,:]==1)
    mask=mask[0,:,:].drop('time')
    return mask


def subset_data(ds_tem,ireg,boxes):
    if (ireg<5) or (ireg>6):
        tem = ds_tem.sel(lon=slice(boxes.lon_max[ireg],boxes.lon_min[ireg]),
                         lat=slice((boxes.lat_cen[ireg]-2.5),(boxes.lat_cen[ireg]+2.5)))
        ds_mean = tem.mean({'lat','lon'},keep_attrs=True)
    else:
        if ireg==5:
            rlon=-165
        if ireg==6:
            rlon=-135
        boxes_cen = get_center_chelle(rlon)
        datasets=[]
        for tinc in ds_tem.time:
            cen_lat=boxes_cen.lat.sel(time=tinc,method='nearest').data
#            cen_lat=boxes_cen.latitude.sel(yr=tinc,method='nearest').data
            tem=ds_tem.sel(time=tinc,
                           lon=slice(rlon-5,rlon+5),
                           lat=slice(cen_lat-2.5,cen_lat+2.5))
            mn_new = tem.mean({'lat','lon'},keep_attrs=True)
            datasets.append(mn_new)
        ds_mean = xr.concat(datasets, dim='time')
        ds_mean.load()
    return ds_mean

In [ ]:
dir_timeseries='F:/data/NASA_biophysical/timeseries_data/'
date_start,date_end,date_end2='1993-01-01','2018-12-31','2019-10-31'
darray = ['oscar','godas_mld','ccmp','cmc_sst','cmem_sss','aviso']
mask=get_mask()
itype=0
dtype = darray[itype]
print('getting data:',dtype)
#v6 uses groupby instead of creating climatology first
ds= get_monthly_data(dtype)
mask2=mask.interp(lat=ds.lat,lon=ds.lon,method='nearest')
ds.coords['mask']=mask2
ds=ds.where(ds.mask==1)

ds=ds.sel(time=slice(date_start,date_end2))
ds_tem=ds #.sel(time=slice(date_start,date_end))
print('got data',itype)
ds.coords['lon'] = np.mod(ds['lon'], 360)
ds = ds.sortby(ds.lon)
ds = ds.sel(lat=slice(30,63),lon=slice(180,250))


#make monthly climatologies

ds2 = ds.groupby('time.month').mean()
ds2 = ds2.load()


In [ ]:
#really hacky way to change how seasons defined and groupby and weight average
dpm=[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
mm = [1,2,3,4,5,6,7,8,9,10,11,12]
month_length = xr.DataArray(dpm,dims=('month'),coords={'month':mm})
weights = month_length.isel(month=slice(0,3))
#winter
data = ds2.isel(month=slice(0,3))
weights = month_length.isel(month=slice(0,3))
ds3= (weights*data).sum(dim='month') / weights.sum()
#spring
data = ds2.isel(month=slice(3,6))
weights = month_length.isel(month=slice(3,6))
ds4= (weights*data).sum(dim='month') / weights.sum()
#summer
data = ds2.isel(month=slice(6,9))
weights = month_length.isel(month=slice(6,9))
ds5= (weights*data).sum(dim='month') / weights.sum()
#fall
data = ds2.isel(month=slice(9,12))
weights = month_length.isel(month=slice(9,12))
ds6= (weights*data).sum(dim='month') / weights.sum()
ds_season = xr.concat([ds3,ds4,ds5,ds6], 'season')

In [ ]:
ds_season.u[0,:,:].plot()

In [ ]:
#figure 1 for marisol's paper
# Combine 1D latitude and longitudes into a 2D grid of locations
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

smon=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Nov','Dec']

for imon in range(11):
    lon_2d, lat_2d = np.meshgrid(ds2.lon, ds2.lat)
    u_wind=ds2.u[imon,:,:]
    v_wind=ds2.v[imon,:,:]
    spd=np.sqrt(u_wind**2+v_wind**2)
    #wind = plt.quiver(lon_2d, lat_2d, u_wind, v_wind,color='b')

    fig = plt.figure(figsize=(14,7))
    ax = fig.add_subplot(1, 1, 1,
                         projection=ccrs.PlateCarree())
    ax.pcolormesh(lon_2d, lat_2d, spd+3,cmap='pink',vmin=0,vmax=1) #hack to make ocean white
    ax.quiver(lon_2d, lat_2d, u_wind, v_wind,color='blue',scale=8,alpha=.8)
    ax.coastlines(resolution='50m')
    ax.stock_img()
    ax.set_xlim(-175,-120)
    ax.set_ylim(35,61)
    box_color=['fuchsia','lime','yellow','crimson']
    box_label=['NPC','Bif','CalC','AkC','Ak2']
    for ii in range(0,4):
        ii2=ii
        if ii==2:
            ii2=7
      #  if ii==5:
       #     continue
        x=[boxes.lon_min[ii2],boxes.lon_max[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]-2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii],label=box_label[ii])
        x=[boxes.lon_max[ii2],boxes.lon_min[ii2]]
        y=[boxes.lat_cen[ii2]+2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
        x=[boxes.lon_min[ii2],boxes.lon_min[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
        x=[boxes.lon_max[ii2],boxes.lon_max[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
    ax.text(-135,59.5,'Month = ' + smon[imon],size=16)
    ax.legend(fontsize=16,loc='upper right')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlabels_bottom = True
    gl.ylabels_left = True
    gl.xlines = False
    gl.xlocator = mticker.FixedLocator([-170,-160,-150,-140,-130,-120])
    gl.ylocator = mticker.FixedLocator([35,40,45,50,55,60])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 16, 'weight': 'bold'}
    gl.ylabel_style = {'size': 16, 'weight': 'bold'}
    figout='F:/data/NASA_biophysical/monthly_curr_figures/boxes'+str(imon).zfill(2)+'.png'
    fig.savefig(figout, dpi=100)

In [ ]:
#figure 1 for marisol's paper SEASONAL
# Combine 1D latitude and longitudes into a 2D grid of locations
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

#smon=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Nov','Dec']
smon=['JFM','AMJ','JAS','OND'] #ds3.season
ds3=ds_season
for imon in range(4):
    lon_2d, lat_2d = np.meshgrid(ds2.lon, ds2.lat)
    u_wind=ds3.u[imon,:,:].where(ds3.mask==1)
    v_wind=ds3.v[imon,:,:].where(ds3.mask==1)
    spd=np.sqrt(u_wind**2+v_wind**2)
    #wind = plt.quiver(lon_2d, lat_2d, u_wind, v_wind,color='b')

    fig = plt.figure(figsize=(14,7))
    ax = fig.add_subplot(1, 1, 1,
                         projection=ccrs.PlateCarree())
    ax.pcolormesh(lon_2d, lat_2d, spd+3,cmap='pink',vmin=0,vmax=1) #hack to make ocean white
    ax.quiver(lon_2d, lat_2d, u_wind, v_wind,color='blue',scale=8,alpha=.8)
    ax.coastlines(resolution='50m')
    ax.stock_img()
    ax.set_xlim(-175,-120)
    ax.set_ylim(35,61)
    box_color=['fuchsia','lime','yellow','crimson']
    box_label=['NPC','Bif','CalC','AkC','Ak2']
    for ii in range(0,4):
        ii2=ii
        if ii==2:
            ii2=7
      #  if ii==5:
       #     continue
        x=[boxes.lon_min[ii2],boxes.lon_max[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]-2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii],label=box_label[ii])
        x=[boxes.lon_max[ii2],boxes.lon_min[ii2]]
        y=[boxes.lat_cen[ii2]+2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
        x=[boxes.lon_min[ii2],boxes.lon_min[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
        x=[boxes.lon_max[ii2],boxes.lon_max[ii2]]
        y=[boxes.lat_cen[ii2]-2.5,boxes.lat_cen[ii2]+2.5]
        line = ax.plot(x, y, linewidth=4, color=box_color[ii])
    ax.text(-135,59.5,'Month = ' + smon[imon],size=16)
    ax.legend(fontsize=16,loc='upper right')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlabels_bottom = True
    gl.ylabels_left = True
    gl.xlines = False
    gl.xlocator = mticker.FixedLocator([-170,-160,-150,-140,-130,-120])
    gl.ylocator = mticker.FixedLocator([35,40,45,50,55,60])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 16, 'weight': 'bold'}
    gl.ylabel_style = {'size': 16, 'weight': 'bold'}
    figout='F:/data/NASA_biophysical/monthly_curr_figures/boxes_season'+str(imon).zfill(2)+'.png'
    fig.savefig(figout, dpi=100)